# Experiments KuHar

This notebook will perform basic experiments on the balanced KuHar dataset with the following steps:
1. Quick load train, test and validation CSV subsets from the balanced KuHar dataset using `PandasDatasetsIO` helper
2. Subclassing the `Dataset` interface using `PandasMultiModalDataset`
3. Apply the fourier transform on the dataset
4. Train SVM, KNN and Random Forest classification models in both time and frequency domains
5. Evaluate those models in both domains

The experiments will evaluate the performance of SVM, KNN and RF models on the balanced KuHar dataset in both time and frequency domains.

## Common imports and definitions

In [1]:
from pathlib import Path  # For defining dataset Paths
import sys                # For include librep package

# This must be done if librep is not installed via pip,
# as this directory (examples) is appart from librep package root
sys.path.append("..")

# Third party imports
import pandas as pd
import numpy as np

# Librep imports
from librep.utils.dataset import PandasDatasetsIO          # For quick load train, test and validation CSVs
from librep.datasets.multimodal import PandasMultiModalDataset # Wrap CSVs to librep's `Dataset` interface

2022-08-29 23:35:11.224640: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-29 23:35:11.224662: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Loading data

In [2]:
# Path for KuHar balanced view with the same activities (and labels numbers)
# It is assumed that the directory will contain (train.csv, test.csv and validation.csv)
kuhar_dataset_path = Path("../data/views/KuHar/balanced_view")

Once paths is defined, we can load the CSV as pandas dataframes

In [3]:
# Kuhar dataframes
kh_train, kh_validation, kh_test = PandasDatasetsIO(kuhar_dataset_path).load()

Letś take a look in the train dataframes

In [4]:
kh_train.head()

,Unnamed: 0,accel-x-0,accel-x-1,accel-x-2,accel-x-3,accel-x-4,accel-x-5,accel-x-6,accel-x-7,accel-x-8,...,gyro-z-299,accel-start-time,gyro-start-time,accel-end-time,gyro-end-time,activity code,length,serial,index,user
0,0,-0.007251,-0.016431,-0.001900,-0.020529,-0.027133,-0.019558,-0.014525,-0.002541,0.016369,...,0.002956,23.235,23.223,26.260,26.249,0,300,1,2100,1051
1,1,-0.008128,-0.006837,0.008597,0.014337,0.006973,0.003250,-0.005086,-0.014379,-0.007034,...,0.001709,56.292,56.292,59.245,59.245,0,300,1,5700,1037
2,2,-0.033081,-0.037222,-0.043654,-0.038211,0.014246,0.063478,0.043582,-0.013673,-0.029928,...,0.002550,27.268,27.267,30.290,30.291,0,300,1,2700,1075
3,3,-0.009740,-0.016656,0.002454,-0.023503,-0.023115,-0.006241,0.017415,0.014765,0.019231,...,0.002969,39.421,39.420,42.441,42.440,0,300,6,3900,1008
4,4,0.029113,0.042745,0.017337,-0.015903,-0.027398,-0.010438,-0.026766,-0.013397,-0.008499,...,0.006943,23.703,23.703,26.656,26.656,0,300,1,2400,1038


## Creating a Librep dataset from pandas dataframes

In [5]:
# Kuhar features to select
features = [
    "accel-x",
    "accel-y",
    "accel-z",
    "gyro-x",
    "gyro-y",
    "gyro-z"
]

# Creating the datasets

# Train
kh_train_dataset = PandasMultiModalDataset(
    kh_train,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)

# Validation
kh_validation_dataset = PandasMultiModalDataset(
    kh_validation,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)

# Test
kh_test_dataset = PandasMultiModalDataset(
    kh_test,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)

## Inspect sample

In [6]:
# Lets print the first sample of kh_train dataset.
# Is a tuple, with an vector of 1800 elements as first element and the label as second
x = kh_train_dataset[0]
print(x)

(array([-0.00725079, -0.01643086, -0.00189972, ...,  0.00295611,
        0.00295611,  0.00295611]), 0)


In [7]:
# Inspecting sample
print(f"The sample 0: {x[0]}")
print(f"Shape of sample 0: {x[0].shape}")
print(f"The label of sample 0: {x[1]}")

The sample 0: [-0.00725079 -0.01643086 -0.00189972 ...  0.00295611  0.00295611
  0.00295611]
Shape of sample 0: (1800,)
The label of sample 0: 0


## Fourier Transform

In [8]:
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT

In [9]:
fft_transform = FFT(centered = True)
transformer = TransformMultiModalDataset(transforms=[fft_transform], new_window_name_prefix="fft.")

### Use FFT in Kuhar

In [10]:
kh_train_dataset_fft = transformer(kh_train_dataset)
kh_validation_dataset_fft = transformer(kh_validation_dataset)
kh_test_dataset_fft = transformer(kh_test_dataset)

In [11]:
kh_train_dataset[:][0]

array([[-7.2507860e-03, -1.6430855e-02, -1.8997192e-03, ...,
         2.9561063e-03,  2.9561063e-03,  2.9561063e-03],
       [-8.1281660e-03, -6.8368910e-03,  8.5973740e-03, ...,
         3.8299560e-03,  2.7618408e-03,  1.7089844e-03],
       [-3.3081055e-02, -3.7221910e-02, -4.3654440e-02, ...,
         3.6152380e-03,  2.5499745e-03,  2.5499745e-03],
       ...,
       [ 3.5006000e-01,  2.6262000e-01, -1.7055000e-01, ...,
         1.5103000e-01,  1.5636000e-01,  1.6275000e-01],
       [-1.6479000e+00, -1.6806000e+00, -1.3551000e+00, ...,
         5.5797000e-01,  5.4838000e-01,  5.3240000e-01],
       [-6.2135534e+00, -7.1491690e+00, -8.0634130e+00, ...,
         1.2653333e-01,  4.5037340e-01,  7.2521144e-01]])

In [12]:
kh_train_dataset_fft[:][0]

array([[1.04461665e-01, 5.60405443e-01, 3.01849883e-01, ...,
        1.65037529e-02, 1.63466728e-02, 1.24300339e-02],
       [7.69014383e-02, 4.12171994e-01, 2.09383077e-01, ...,
        2.63120409e-03, 9.48805261e-03, 5.18175730e-03],
       [2.60636336e-01, 2.41408098e-01, 4.46786522e-01, ...,
        3.69022318e-03, 1.16167546e-02, 1.45708099e-03],
       ...,
       [1.23297140e+01, 1.48896039e+02, 1.29219421e+02, ...,
        2.89959395e-01, 2.99217590e-01, 3.00465532e-01],
       [1.56351597e+01, 1.06429741e+02, 4.65818805e+01, ...,
        4.51856425e-01, 3.94662803e-01, 4.98044870e-01],
       [7.44451167e+01, 3.31692910e+01, 4.29451498e+01, ...,
        1.67725714e+00, 1.48620599e+00, 1.40784994e+00]])

## Train Random Forest classifier

In [13]:
from librep.utils.workflow import SimpleTrainEvalWorkflow, MultiRunWorkflow
from librep.estimators import RandomForestClassifier
from librep.metrics.report import ClassificationReport
import yaml

reporter = ClassificationReport(use_accuracy=True, use_f1_score=True, use_classification_report=False, use_confusion_matrix=False, plot_confusion_matrix=False)
experiment = SimpleTrainEvalWorkflow(estimator=RandomForestClassifier, estimator_creation_kwags ={'n_estimators':100} , do_not_instantiate=False, do_fit=True, evaluator=reporter)
multi_run_experiment = MultiRunWorkflow(workflow=experiment, num_runs=3, debug=False)

In [14]:
combined_train_dset = PandasMultiModalDataset(
    pd.concat([kh_train, kh_validation]),
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)


result = multi_run_experiment(combined_train_dset, kh_test_dataset)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661816124.048003
    result:
    -   accuracy: 0.7037037037037037
        f1 score (macro): 0.690764546204045
        f1 score (micro): 0.7037037037037037
        f1 score (weighted): 0.7166428612033623
    run id: 1
    start: 1661816114.3191776
    time taken: 9.728825330734253
-   end: 1661816133.8004167
    result:
    -   accuracy: 0.6984126984126984
        f1 score (macro): 0.6858942569494191
        f1 score (micro): 0.6984126984126984
        f1 score (weighted): 0.7109311398759778
    run id: 2
    start: 1661816124.0480049
    time taken: 9.752411842346191
-   end: 1661816143.530765
    result:
    -   accuracy: 0.7195767195767195
        f1 score (macro): 0.7092780195012945
        f1 score (micro): 0.7195767195767196
        f1 score (weighted): 0.7298754196521444
    run id: 3
    start: 1661816133.8004184
    time taken: 9.7303466796875



In [15]:
combined_train_dset_fft = transformer(combined_train_dset)
    
result = multi_run_experiment(combined_train_dset_fft, kh_test_dataset_fft)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1661816149.6383033
    result:
    -   accuracy: 0.7883597883597884
        f1 score (macro): 0.7840201443680451
        f1 score (micro): 0.7883597883597884
        f1 score (weighted): 0.7926994323515316
    run id: 1
    start: 1661816143.8016706
    time taken: 5.83663272857666
-   end: 1661816155.4896622
    result:
    -   accuracy: 0.7804232804232805
        f1 score (macro): 0.7740330091070471
        f1 score (micro): 0.7804232804232806
        f1 score (weighted): 0.7868135517395137
    run id: 2
    start: 1661816149.638305
    time taken: 5.8513572216033936
-   end: 1661816161.3321176
    result:
    -   accuracy: 0.7486772486772487
        f1 score (macro): 0.739769881601573
        f1 score (micro): 0.7486772486772487
        f1 score (weighted): 0.7575846157529242
    run id: 3
    start: 1661816155.489664
    time taken: 5.842453479766846



In [16]:
combined_train_dset_fft.window_slices, combined_train_dset_fft.window_names, combined_train_dset_fft.num_windows

([(0, 150), (150, 300), (300, 450), (450, 600), (600, 750), (750, 900)],
 ['fft.accel-x',
  'fft.accel-y',
  'fft.accel-z',
  'fft.gyro-x',
  'fft.gyro-y',
  'fft.gyro-z'],
 6)